# Decision Function and Prediction Probability

Follow _Introduction to Machine Learning_ 
- [Chapter 2](https://github.com/amueller/introduction_to_ml_with_python/blob/master/02-supervised-learning.ipynb) **Section 2.7 Uncertanty estimates from classifiers** (p.121 - 125)
- [Chapter 5](https://github.com/amueller/introduction_to_ml_with_python/blob/master/05-model-evaluation-and-improvement.ipynb) **Section 5.3.2 Metrics for Binary Classification**  - taking uncertainty into account (p.292)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import mglearn

## Uncertainty estimates from classifiers
>... the ability of classifiers to provide uncertainty estimates of predictions. Often, you are not only interested in which class a classifier predicts for a certain test point, but also how certain it is that this is the right class.

>There are two different functions in scikit-learn that can be used to obtain uncertainty estimates from classifiers: decision_function and predict_proba. Most (but not all) classifiers have at least one of them, and many classifiers have both.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(noise=0.25, factor=0.5, random_state=1)

# we rename the classes "blue" and "red" for illustration purposes:
y_named = np.array(["blue", "red"])[y]

# we can call train_test_split with arbitrarily many arrays;
# all will be split in a consistent manner
X_train, X_val, y_train_named, y_val_named, y_train, y_val = \
    train_test_split(X, y_named, y, random_state=0)

# build the gradient boosting model
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train_named)

### Decision function
For each sample, the decision function produces a value that indicates how certain the classfier is that this sample belongs to a class.

The range of the decision function can be arbitrary, and spans positive and negative values. However, by default a value of 0 determines the decision boundary.

In [ ]:
print("X_val.shape:", X_val.shape)
print("Decision function shape:",
      gbrt.decision_function(X_val).shape)

In [ ]:
# show the first few entries of decision_function
print("Decision function:", gbrt.decision_function(X_val)[:6])

>We can recover the prediction by looking only at the sign of the decision function:

In [ ]:
print("Thresholded decision function:\n",
      gbrt.decision_function(X_val) > 0)
print("Predictions:\n", gbrt.predict(X_val))

>For binary classification, the “negative” class is always the first entry of the classes_ attribute, and the “positive” class is the second entry of classes_. So if you want to fully recover the output of predict, you need to make use of the classes_ attribute:

In [ ]:
# make the boolean True/False into 0 and 1
greater_zero = (gbrt.decision_function(X_val) > 0).astype(int)
# use 0 and 1 as indices into classes_ (classes is ['blue', 'red'])
pred = gbrt.classes_[greater_zero]
# pred is the same as the output of gbrt.predict
print("pred is equal to predictions:",
      np.all(pred == gbrt.predict(X_val)))

How does the indexing work here to recover 'red' and 'blue'?

In [ ]:
gbrt.classes_

In [ ]:
pred

In [ ]:
greater_zero

In [ ]:
np.array(['blue', 'red'])[np.array([0, 0, 1])]

### What is the range of the decision function?

In [ ]:
decision_function = gbrt.decision_function(X_val)
print("Decision function minimum: {:.2f} maximum: {:.2f}".format(
      np.min(decision_function), np.max(decision_function)))

>This arbitrary scaling makes the output of decision_function often hard to interpret.

>In the following example we plot the decision_function for all points in the 2D plane using a color coding, next to a visualization of the decision boundary, as we saw earlier. We show training points as circles and test data as triangles

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
mglearn.tools.plot_2d_separator(gbrt, X, ax=axes[0], alpha=.4,
                                fill=True, cm=mglearn.cm2)
scores_image = mglearn.tools.plot_2d_scores(gbrt, X, ax=axes[1],
                                            alpha=.4, cm=mglearn.ReBl)

for ax in axes:
    # plot training and validation points
    mglearn.discrete_scatter(X_val[:, 0], X_val[:, 1], y_val,
                             markers='^', ax=ax)
    mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train,
                             markers='o', ax=ax)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")
cbar = plt.colorbar(scores_image, ax=axes.tolist())
cbar.set_alpha(1)
axes[0].legend(["Validation class 0", "Validation class 1", "Train class 0",
                "Train class 1"], ncol=4, loc=(.1, 1.1))

### Prediction probability

For each sample, the prediction probability indicates how certain the classfier is that this sample belongs to a class.

The range of the probability is between 0 and 1. By default a probability of 0.5 determines the decision boundary.

In [ ]:
print("Shape of probabilities:", gbrt.predict_proba(X_val).shape)

We get two probabilities for each sample, one for the negative and one for the positive class.

In [ ]:
# show the first few entries of predict_proba
print("Predicted probabilities:")
print(gbrt.predict_proba(X_val[:6]))

When plotting the probability, we choose the positive class.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
    
mglearn.tools.plot_2d_separator(
    gbrt, X, ax=axes[0], alpha=.4, fill=True, cm=mglearn.cm2)
scores_image = mglearn.tools.plot_2d_scores(
    gbrt, X, ax=axes[1], alpha=.5, cm=mglearn.ReBl, function='predict_proba')

for ax in axes:
    # plot training and Validation points
    mglearn.discrete_scatter(X_val[:, 0], X_val[:, 1], y_val,
                             markers='^', ax=ax)
    mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train,
                             markers='o', ax=ax)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")
# don't want a transparent colorbar
cbar = plt.colorbar(scores_image, ax=axes.tolist())
cbar.set_alpha(1)
axes[0].legend(["Validation class 0", "Validation class 1", "Train class 0",
                "Train class 1"], ncol=4, loc=(.1, 1.1))

## Changing decision thresholds
>As we discussed in Chapter 2, most classifiers provide a decision_function or a predict_proba method to assess degrees of certainty about predictions. Making predictions can be seen as thresholding the output of decision_function or predict_proba at a certain fixed point—in binary classification we use 0 for the decision function and 0.5 for predict_proba.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

X, y = make_blobs(n_samples=(400, 50), cluster_std=[7.0, 2],
                  random_state=22)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)
svc = SVC(gamma=.05).fit(X_train, y_train)

For decision functions, positive values indicate positive class; negative values negative class by default.

Applying a threshold that is lower than 0 will move the decision boundary in the direction of the negative class, hence, growing the positive class region:

In [ ]:
mglearn.plots.plot_decision_threshold()

Be decreasing the threshold, we therefore predict more positive classes. We should be able to catch more actual positives, not make false negative mistakes, and recall should go up.

On the other hand, we will make more false positive mistakes now, as we are predicting more positive class samples. Precision will drop, we are less often correct.

Let's have a look at the classification reports and confusion matrix with and without threshold adjustment:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, svc.predict(X_val)))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_val, 
                                        svc.predict(X_val), 
                                        display_labels=['blue', 'red'],
                                        cmap='rocket',
                                        colorbar=False);

In [ ]:
y_pred_lower_threshold = svc.decision_function(X_val) > -0.8

In [ ]:
print(classification_report(y_val, y_pred_lower_threshold))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val, 
                                        y_pred_lower_threshold, 
                                        display_labels=['blue', 'red'],
                                        cmap='rocket',
                                        colorbar=False);

Adjusting the threshold in the other direction, more positive, should have the opposite effect:
- recall goes down
- precision goes up

In [ ]:
y_pred_higher_threshold = svc.decision_function(X_val) > 0.8

In [ ]:
print(classification_report(y_val, y_pred_higher_threshold))

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val, 
                                        y_pred_higher_threshold, 
                                        display_labels=['blue', 'red'],
                                        cmap='rocket',
                                        colorbar=False);